# Train a custom YOLO model

### Install Packages

In [ ]:
# %pip install roboflow
# %pip install utralytics
# %pip install python-dotenv

### Import Packages

In [1]:
from roboflow import Roboflow
from ultralytics import YOLO
from dotenv import load_dotenv
from pathlib import Path
import torch

import os

In [2]:
# Load enviroment variables
dotenv_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path)

ROBOFLOW_API = os.getenv("ROBOFLOW_API")
if not ROBOFLOW_API:
    raise ValueError("No roboflow API found in .env")

cwd = Path.cwd()
project_dir = Path.cwd().parent

## 1. Import training dataset from roboflow

In [3]:
rf_workspace = "test-vlslb"
rf_project = "car-owtrd-2we06"
rf_project_version = 1

rf = Roboflow(api_key=ROBOFLOW_API) #Place download code from roboflow
project = rf.workspace(rf_workspace).project(rf_project)
version = project.version(rf_project_version)

dataset_path = cwd / "model_training_data" / "training_data_00"
overwrite_dataset = False
dataset = version.download("yolov12", dataset_path, overwrite_dataset)

loading Roboflow workspace...
loading Roboflow project...


## 2. Train the model

Batch size info:

| Batch size | GPU Usage |
|------------|-----------|
| 5          | >8GB      |

In [8]:
torch.cuda.empty_cache()

#define the model based on existing Yolo model (in this case the segmentation model)
models_path = cwd / "models"
pretrained_model_path =  models_path / "yolo12x.pt"
new_model_path = models_path / "00_trained_model"
model = YOLO(pretrained_model_path)
print(model.info())

#define variables
dataset = dataset_path / "data.yaml"
epoch_size = 30
image_size = 640
batch_size = 5

#train the model based on the labeled data from Roboflow
# model.train(data=dataset ,  epochs=epoch_size, batch=batch_size, imgsz=image_size, project=new_model_path)
best_model = YOLO(new_model_path / "train2" / "weights" / "best.pt")

YOLOv12x summary: 488 layers, 59,216,928 parameters, 0 gradients, 200.3 GFLOPs
(488, 59216928, 0, 200.3276288)


## 3. Validate the model

In [10]:
# Show performance metrics
metrics = best_model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95(B)
metrics.box.map50  # map50(B)
metrics.box.map75  # map75(B)
metrics.box.maps  # a list contains map50-95(B) of each category

Ultralytics 8.3.199  Python-3.13.0 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
val: Fast image access  (ping: 0.20.0 ms, read: 1115.6383.3 MB/s, size: 538.1 KB)
val: Scanning C:\Users\tomer\OneDrive\Home\Projects\Personal\satellite_imagery_interpreter\AI_Model_Training\model_training_data\training_data_00\valid\labels.cache... 22 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 22/22 31.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 2/2 0.5it/s 4.1s12.3s
                   all         22        471      0.968      0.979      0.993      0.729
Speed: 0.3ms preprocess, 57.3ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to C:\Users\tomer\OneDrive\Home\Projects\Personal\satellite_imagery_interpreter\AI_Model_Training\runs\detect\val3


array([    0.72852])